In [4]:
from datasets import load_dataset

dataset = load_dataset("wikitext", "wikitext-2-v1")

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Dataset wikitext downloaded and prepared to /home/leander/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
import nltk

In [31]:
train_sentences = []

for text in dataset["train"]:
    train_sentences.extend(nltk.sent_tokenize(text["text"]))
    
train_sentences = [sentence.lower().strip().split() for sentence in train_sentences if sentence.strip()]
sentence_lengths = [len(sentence) for sentence in train_sentences]

min_length, max_length = np.quantile(sentence_lengths, q=(0.05, 0.95))
min_length = round(min_length)
max_length = round(max_length)

train_sentences = [sentence for sentence in train_sentences if min_length <= len(sentence) <= max_length]

In [32]:
from collections import Counter

In [33]:
all_tokens = []

for sentence in train_sentences:
    all_tokens.extend(sentence)
    
token_counts = Counter(all_tokens)

In [38]:
vocab = set(list(sorted(token_counts.keys(), key=token_counts.get, reverse=True))[:10000])

In [39]:
import os

In [40]:
os.makedirs("data", exist_ok=True)

In [41]:
def write_split(split_name: str):
    with open(f"data/{split_name}.txt", "w") as sf:
        for text in dataset[split_name]:
            text = text["text"].lower().strip()
            
            sentences = nltk.sent_tokenize(text)
            for sentence in sentences:
                sentence = sentence.split()
                if not sentence or not (min_length <= len(sentence) <= max_length):
                    continue
                
                sentence = [(token if token in vocab else "<unk>") for token in sentence]
                sf.write(" ".join(sentence) + "\n")

In [42]:
write_split("train")
write_split("test")
write_split("validation")